In [14]:
import yfinance as yf
import numpy as np
import pandas as pd
import tweepy as tw
from datetime import datetime, timedelta

import emot
import nltk # sentiment library
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('vader_lexicon') # download vader lexicon
nltk.download("stopwords")
nltk.download("wordnet")

from nltk.sentiment import SentimentIntensityAnalyzer as SIA

from textblob import TextBlob

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
#Grab cryptocurrency price

btc_data = yf.download(tickers='BTC-USD', period='1mo', interval='1d')
eth_data = yf.download(tickers='ETH-USD', period='1mo', interval='1d')

btc_df = pd.DataFrame(btc_data)
eth_df = pd.DataFrame(eth_data)

btc_df.to_csv("btc_price.csv")
eth_df.to_csv("eth_price.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [18]:
#Grab tweets
consumer_key= 'Ou9XaMR4qgy3KGr7cSfvhjnYl'
consumer_secret= 'kTl5Vs6918QYgEkufJVPbpfbuyFAyyjYfkRcl2hAL5BjGyqxyj'
access_token= '2991366318-LhMbjkJGvNkmO5bwSzVfMhZQWl19dzoPdgOWD8V'
access_token_secret= 'qmRpWmw2UHP3VPYOoodeOIt2s8liaGnEaePDSF9JtFIYI'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_words = "#btc OR #bitcoin OR bitcoin OR btc"

today = datetime.utcnow().date()- timedelta(days=1)

yesterday = today - timedelta(days=1)

date_since = datetime.today().strftime('%Y-%m-%d')

tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=yesterday, until=today).items()

In [19]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english", ignore_stopwords=True)
lemmatizer = WordNetLemmatizer()
tokenzr = RegexpTokenizer('\s+', gaps = True)
    
def stem_tokenize(text):
    tokenized_text = tokenzr.tokenize(text.lower())
    words = [lemmatizer.lemmatize(w) for w in tokenized_text if w not in stop_words]
    stem_text = " ".join([stemmer.stem(i) for i in words])

    return stem_text #converted emoji to unicode description of emoji is return as string text


def regex_tweet(tweet):
    tweet = ' '.join(re.sub("(\$[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    tweet = ' '.join(re.sub("[\[\]]", " ", tweet).split())
    return tweet


def clean_mean(val):
    return val.replace('_', ' ').replace('-', ' ').replace(':', ' ')


def convert_emojicon(text):
    for emoti in emot.emo_unicode.EMOTICONS:
        if emoti in text:
            text = text.replace(emoti, clean_mean(emot.emo_unicode.EMOTICONS.get(emoti, '')))
            
    for emoti in emot.emo_unicode.UNICODE_EMO:
        if emoti in text:
            text = text.replace(emoti, clean_mean(emot.emo_unicode.UNICODE_EMO.get(emoti, '')))
            
    for emoti in emot.emo_unicode.EMOTICONS_EMO:
        if emoti in text:
            text = text.replace(emoti, clean_mean(emot.emo_unicode.EMOTICONS_EMO.get(emoti, '')))
            
    return text

sia = SIA()

count = 0
sentiment_counter = 0
for tweet in tweets:
    tweet_id = tweet.id
    tweet_time = tweet.created_at
    tweet_username = tweet.author.screen_name
    
    tweet_hashtags = tweet.entities['hashtags']
    hashtags = []
    for hashtag in tweet_hashtags:
        hashtags.append(hashtag['text'])
        
    cashtags = ""
    link = "https://twitter.com/{username}/status/{id}".format(username = tweet_username, id = tweet_id)
    
    tweet_text = tweet.text
    tweet_text = convert_emojicon(tweet_text)
    
    tweet_sentiment = TextBlob(tweet_text).sentiment.polarity
    sentiment_counter += tweet_sentiment
        
    print(tweet_id, tweet_time, tweet_username, hashtags, link, tweet_text, tweet_sentiment)
    count += 1
    if count > 5:
        break
    

1385745298270552065 2021-04-23 23:59:59 MustafaRaaR [] https://twitter.com/MustafaRaaR/status/1385745298270552065 Also, the value of the coins can drop drastically in a single day. As much as 15%. httpsSkeptical, annoyed, undecided, uneasy or hesitant/t.co/tDf5suQLaughing, big grin or laugh with glassesX 0.005714285714285705
1385745298127867904 2021-04-23 23:59:59 wokkrr [] https://twitter.com/wokkrr/status/1385745298127867904 RT @CryptoHunterGon: Perfect hold of 4H Pi line and daily 60 EMA for $woo 

Is it really this easy? 

BTC gonna have to hold current lows e… 0.32666666666666666
1385745295758307328 2021-04-23 23:59:59 LinkFr0g [] https://twitter.com/LinkFr0g/status/1385745295758307328 @gross_bit What you fail to account for is LT sellers that may be rotating the money from BTC (selling) into alts (… httpsSkeptical, annoyed, undecided, uneasy or hesitant/t.co/SGP4RunUed -0.45
1385745283577905153 2021-04-23 23:59:56 DaniySuniy ['BSV'] https://twitter.com/DaniySuniy/status/138574528